In [1]:
import requests
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-setuid-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from time import sleep
import random
from tqdm import tqdm
from tqdm.auto import trange
import pandas as pd

# 플레이스토어 크롤링

## 무한 스크롤 함수

In [ ]:
chrome_driver = 'chromedriver.exe'
url = 'https://play.google.com/store/apps/details?id=com.mhq.im'

In [7]:
def scroll(modal):
    try:
        #스크롤 높이 받아기
        last_height = driver.execute_script('return arguments[0].scrollHeight', modal)
        while True:
            pause_time = random.uniform(0.5, 0.8)
            #최하단까지 스크롤
            driver.execute_script('arguments[0].scrollTo(0, arguments[0].scrollHeight);', modal)
            #페이지 로딩 대기
            time.sleep(pause_time)
            #무한 스크롤 동작을 위해 살짝위로 스크롤
            driver.execute_script('arguments[0].scrollTo(0, arguments[0].scrollHeight-50);', modal)
            time.sleep(pause_time)
            #스크롤 높이 새롭게 받아오기
            new_height = driver.execute_script('return arguments[0].scrollHeight', modal)
            
            try:
                #'더보기' 버튼 있을 경우 클릭
                all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/')
            except:
                #스크롤 완료 경우
                if new_height == last_height:
                    print('스크롤 완료')
                    break
                last_height = new_height
        
    except Exception as e:
        print('에러 발생:', e)

## 데이터 크롤링

In [23]:
# 크롬 드라이버 세팅
driver = webdriver.Chrome(chrome_driver, options = chrome_options)
# 페이지열기
driver.get(url)
#페이지 로딩 대기
wait = WebDriverWait(driver, 5)

# '리뷰 모두 보기' 버튼 렌더링 확인
all_review_button_xpath = '/html/body/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[3]/section/div/div/div[5]/div/div/button/span'
button_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_button_xpath)))
# '리뷰 모두 보기' 버튼 클릭
driver.find_element_by_xpath(all_review_button_xpath).click()

# '리뷰 모두 보기' 페이지 렌더링 대기
all_review_page_xpath = '/html/body/div[4]/div[2]/div/div/div/div/div[2]'
page_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_page_xpath)))

#페이지 무한 스크롤 다운
modal = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
scroll(modal)

스크롤 완료


## HTML Parsing

In [24]:
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

## 데이터 프레임 변환

In [25]:
#리뷰 데이터 클래스 접근
review_source = soup_source.find_all(class_ = 'RHo1pe')
#리뷰 데이터 저장용 배열
dataset = []
#데이터 넘버링을 위한 변수
review_num = 0
#리뷰 1개씩 접근해 정보 추출
for review in tqdm(review_source):
    review_num += 1
    #리뷰 등록일 데이터 추출
    date_full = review.find_all(class_ = 'bp9Aid')[0].text
    date_year = date_full[0:4] # 연도 데이터 추출
    # 해당 단어가 등장한 인덱스 추출
    year_index = date_full.find('년')
    month_index = date_full.find('월')
    day_index = date_full.find('일')
    
    date_month = str(int(date_full[year_index+1:month_index])) # 월(Month) 데이터 추출
    # 월 정보가 1자리의 경우 앞에 0 붙이기(e.g., 1월 -> 01월)
    if len(date_month) == 1:
        date_month = '0' + date_month
    
    date_day = str(int(date_full[month_index+1:day_index])) # 일(Day) 데이터 추출 
    # 일 정보가 1자리의 경우 앞에 0 붙여줌(e.g., 7일 -> 07일)
    if len(date_day) == 1:
        date_day = '0' + date_day
        
    # 리뷰 등록일 full version은 최종적으로 yyyymmdd 형태로 저장
    date_full = date_year + date_month + date_day
    user_name = review.find_all(class_ = 'X5PpBb')[0].text # 닉네임 데이터 추출
    rating = review.find_all(class_ = "iXRFPc")[0]['aria-label'][10] # 평점 데이터 추출
    content = review.find_all(class_ = 'h3YV2d')[0].text # 리뷰 데이터 추출

    data = {
        "id": review_num, 
        "date": date_full,
        "dateYear": date_year,
        "dateMonth": date_month,
        "dateDay": date_day,
        "rating": rating,
        "userName": user_name,
        "content": content
    }
    dataset.append(data)

100%|██████████████████████████████████████████████████████████████████████████████| 813/813 [00:00<00:00, 1316.47it/s]


In [26]:
review_df = pd.DataFrame(dataset)
review_df

,id,date,dateYear,dateMonth,dateDay,rating,userName,content
0,1,20220301,2022,03,01,1,원배원배,차는 크고 괜찮으나 고객센터 야간에 전화를 받지 않음. 플랫폼만으러 사업하면서 응급...
1,2,20220304,2022,03,04,5,고인석,넓고 깨끗한 콜택시 탑승은 만족하지만 기사분들 고충이 심한듯 고객의 콜을 부르면 목...
2,3,20220226,2022,02,26,4,DY Kim,"항상 잘 이용하고 있습니다. 친절한 지니님들과 콜비없는 빠른 배차에 만족합니다만, ..."
3,4,20220215,2022,02,15,1,민경찬,아니 왜 안드로이드 실행하면 항상 먹통이죠? 업데이트하고 지웠다가 다시깔아도 실행이...
4,5,20220225,2022,02,25,3,성이름,타다 보다 안되는이유 딱하나 렌트카 보다 값싼 카니발 끌면서 할인쿠폰 첫탑승 제외 ...
...,...,...,...,...,...,...,...,...
808,809,20210814,2021,08,14,3,은소빈,괜찮아요!!
809,810,20210319,2021,03,19,5,김봉희,참좋습니다
810,811,20211009,2021,10,09,5,이승민,굿
811,812,20210724,2021,07,24,5,소나기Downpour,굿


In [27]:
review_df.to_csv('review_dataset.csv', encoding = 'utf-8-sig') # csv 파일로 저장

# 앱스토어 크롤링

In [2]:
import xmltodict 
from urllib.request import urlopen 
import json

In [4]:
url = "https://itunes.apple.com/kr/rss/customerreviews/page=1/id=1542967200/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1542967200/sortBy=mostRecent/xml" 
response = urlopen(url).read()

In [36]:
xml = xmltodict.parse(response)
XmlToJson = json.loads(json.dumps(xml))

In [37]:
len(XmlToJson['feed']['entry'])

50

In [34]:
res_dict = []
for j in range(1,5):
    url = "https://itunes.apple.com/kr/rss/customerreviews/page=%i/id=1542967200/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1542967200/sortBy=mostRecent/xml" % j
    response = urlopen(url).read()
    
    xml = xmltodict.parse(response)
    XmlToJson = json.loads(json.dumps(xml))
    
    for i in range(len(XmlToJson['feed']['entry'])):
        res_dict.append({
            'date' : XmlToJson['feed']['entry'][i]['updated'],
            'rating' : int(XmlToJson['feed']['entry'][i]['im:rating']),
            'userName' : XmlToJson['feed']['entry'][i]['author']['name'],
            'title' : XmlToJson['feed']['entry'][i]['title'],
            'content' : XmlToJson['feed']['entry'][i]['content'][0]['#text']
        })
        
res_df = pd.DataFrame(res_dict)

In [45]:
res_df['date'] = pd.to_datetime(res_df['date'], format="%Y-%m-%dT%H:%M:%S-07:00")
res_df['date'] = res_df['date'].dt.date
res_df['platform'] = 'AppStore'
res_df

,date,rating,userName,title,content,platform
0,2022-03-07,5,소나무탬버린,택시충으로서 아이엠택시 진짜 강추,"일반택시에서 느꼇던 불편, 불쾌했던 것들? 이 개선된 서비스인거 같음 작년에 눈 ...",AppStore
1,2022-02-14,5,붕붕씽씽마니아,아이엠 마니아,편하고 친절한 아이엠 애용하고 있어요\n\n차량이 많아져서 다양한곳에서\n만나보고싶어요,AppStore
2,2022-02-13,5,첫차사용자(1),아이엠을 알고 아이엠만 씁니다.,카카오 타다 위주로 탔는데요. 아이엠을 알고 아이엠만 사용합니다. 기사님들이 타회사...,AppStore
3,2022-02-13,5,루나신,편안하고 복불복이 아니여서 좋아요,일단 타면 편하고 말도안거시고 \n제 공간을 확보한 기분이 들어서 더 좋습니다.\n...,AppStore
4,2022-02-13,5,바바라밥바,굿,굿,AppStore
...,...,...,...,...,...,...
195,2021-01-08,5,주식대박,편리하고 깨끗해요,앞으로 더 자주 이용할께요^^,AppStore
196,2020-12-31,5,달릴까코바,요긴하게 이용했습니다,역삼동 사무실에 안전하고 쾌적하게\n탑승하고 필요한 기능 많아서 댓글스르륵\n잇츠굿...,AppStore
197,2020-12-31,5,뭐여오마니,진짜 넓고 쾌적함,콜만 바로바로 잡히면 진짜 최고일듯,AppStore
198,2020-12-30,5,윤석열,좋네요,택시보다 훨씬 낫네요\n\n차만 좀 늘리면 \n\n원할때 어디서든 이용할거같아요,AppStore


In [46]:
res_df.to_csv('app_store_review.csv', encoding='utf-8-sig')

# 데이터 결합

In [61]:
play_store = pd.read_csv('review_dataset.csv', index_col = 0)
app_store = pd.read_csv('app_store_review.csv', index_col = 0 )
app_store = app_store.drop('title', axis=1)
app_store['date'] = pd.to_datetime(app_store['date'])
app_store

,date,rating,userName,content,platform
0,2022-03-07,5,소나무탬버린,"일반택시에서 느꼇던 불편, 불쾌했던 것들? 이 개선된 서비스인거 같음 작년에 눈 ...",AppStore
1,2022-02-14,5,붕붕씽씽마니아,편하고 친절한 아이엠 애용하고 있어요\n\n차량이 많아져서 다양한곳에서\n만나보고싶어요,AppStore
2,2022-02-13,5,첫차사용자(1),카카오 타다 위주로 탔는데요. 아이엠을 알고 아이엠만 사용합니다. 기사님들이 타회사...,AppStore
3,2022-02-13,5,루나신,일단 타면 편하고 말도안거시고 \n제 공간을 확보한 기분이 들어서 더 좋습니다.\n...,AppStore
4,2022-02-13,5,바바라밥바,굿,AppStore
...,...,...,...,...,...
195,2021-01-08,5,주식대박,앞으로 더 자주 이용할께요^^,AppStore
196,2020-12-31,5,달릴까코바,역삼동 사무실에 안전하고 쾌적하게\n탑승하고 필요한 기능 많아서 댓글스르륵\n잇츠굿...,AppStore
197,2020-12-31,5,뭐여오마니,콜만 바로바로 잡히면 진짜 최고일듯,AppStore
198,2020-12-30,5,윤석열,택시보다 훨씬 낫네요\n\n차만 좀 늘리면 \n\n원할때 어디서든 이용할거같아요,AppStore


In [62]:
play_store['platform'] = 'PlayStore'
play_store['date'] = pd.to_datetime(play_store['date'], format='%Y%m%d')
play_store = play_store.drop(['id','dateYear','dateMonth','dateDay'], axis=1)
play_store

,date,rating,userName,content,platform
0,2022-03-01,1,원배원배,차는 크고 괜찮으나 고객센터 야간에 전화를 받지 않음. 플랫폼만으러 사업하면서 응급...,PlayStore
1,2022-03-04,5,고인석,넓고 깨끗한 콜택시 탑승은 만족하지만 기사분들 고충이 심한듯 고객의 콜을 부르면 목...,PlayStore
2,2022-02-26,4,DY Kim,"항상 잘 이용하고 있습니다. 친절한 지니님들과 콜비없는 빠른 배차에 만족합니다만, ...",PlayStore
3,2022-02-15,1,민경찬,아니 왜 안드로이드 실행하면 항상 먹통이죠? 업데이트하고 지웠다가 다시깔아도 실행이...,PlayStore
4,2022-02-25,3,성이름,타다 보다 안되는이유 딱하나 렌트카 보다 값싼 카니발 끌면서 할인쿠폰 첫탑승 제외 ...,PlayStore
...,...,...,...,...,...
808,2021-08-14,3,은소빈,괜찮아요!!,PlayStore
809,2021-03-19,5,김봉희,참좋습니다,PlayStore
810,2021-10-09,5,이승민,굿,PlayStore
811,2021-07-24,5,소나기Downpour,굿,PlayStore


In [64]:
review_total = pd.concat([play_store, app_store])
review_total = review_total.reset_index().drop('index', axis=1)
review_total

,date,rating,userName,content,platform
0,2022-03-01,1,원배원배,차는 크고 괜찮으나 고객센터 야간에 전화를 받지 않음. 플랫폼만으러 사업하면서 응급...,PlayStore
1,2022-03-04,5,고인석,넓고 깨끗한 콜택시 탑승은 만족하지만 기사분들 고충이 심한듯 고객의 콜을 부르면 목...,PlayStore
2,2022-02-26,4,DY Kim,"항상 잘 이용하고 있습니다. 친절한 지니님들과 콜비없는 빠른 배차에 만족합니다만, ...",PlayStore
3,2022-02-15,1,민경찬,아니 왜 안드로이드 실행하면 항상 먹통이죠? 업데이트하고 지웠다가 다시깔아도 실행이...,PlayStore
4,2022-02-25,3,성이름,타다 보다 안되는이유 딱하나 렌트카 보다 값싼 카니발 끌면서 할인쿠폰 첫탑승 제외 ...,PlayStore
...,...,...,...,...,...
1008,2021-01-08,5,주식대박,앞으로 더 자주 이용할께요^^,AppStore
1009,2020-12-31,5,달릴까코바,역삼동 사무실에 안전하고 쾌적하게\n탑승하고 필요한 기능 많아서 댓글스르륵\n잇츠굿...,AppStore
1010,2020-12-31,5,뭐여오마니,콜만 바로바로 잡히면 진짜 최고일듯,AppStore
1011,2020-12-30,5,윤석열,택시보다 훨씬 낫네요\n\n차만 좀 늘리면 \n\n원할때 어디서든 이용할거같아요,AppStore


In [65]:
review_total.to_csv('review_total.csv', encoding='utf-8-sig')